In [1]:
# Task 9: LangGraph + Symbolic Inference + CoT Refinement

!pip install -U langchain langchain-community langgraph faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.5 MB/s eta 0:00:00


In [13]:
# --- Knowledge Base ---
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

import re

kb_facts = """\
animal(dog).
animal(cat).
animal(eagle).
animal(sparrow).
animal(crocodile).
animal(whale).
flies(eagle).
flies(sparrow).
has_tail(dog).
has_tail(cat).
lays_eggs(crocodile).
"""

kb_rules = """\
bird(X) :- animal(X), flies(X).
mammal(X) :- animal(X), has_tail(X).
warm_blooded(X) :- mammal(X).
reptile(X) :- animal(X), lays_eggs(X).
cold_blooded(X) :- reptile(X).
"""

docs = [
    Document(page_content=kb_facts, metadata={"source": "facts"}),
    Document(page_content=kb_rules, metadata={"source": "rules"})
]

splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
kb_chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(kb_chunks, embedding)
retriever = vectorstore.as_retriever()

# --- Symbolic Inference (Backward Chaining) ---
from collections import defaultdict

facts = set()
for line in kb_facts.strip().splitlines():
    pred, arg = line.strip(").").split("(")
    facts.add((pred.strip(), arg.strip()))

rules = defaultdict(list)
for line in kb_rules.strip().splitlines():
    head, body = line.split(":-")
    head_pred, head_var = head.strip(").").split("(")
    body_preds = []
    for pred in body.strip().split(","):
        pred_name, pred_var = pred.strip(").").split("(")
        body_preds.append((pred_name.strip(), pred_var.strip()))
    rules[(head_pred, head_var)].append(body_preds)

def prove(goal_pred, goal_const):
    if (goal_pred, goal_const) in facts:
        return True
    for (head_pred, head_var), rule_bodies in rules.items():
        if head_pred != goal_pred:
            continue
        for body in rule_bodies:
            if all(prove(pred_name, goal_const) for pred_name, _ in body):
                return True
    return False

# --- LangGraph Flow ---
from langgraph.graph import StateGraph, END

# 1. Define state
def create_state(question):
    return {"question": question, "relevant_docs": [], "refined_query": question, "inference_result": None}

# 2. Define nodes
def retrieve_node(state):
    docs = retriever.invoke(state["question"])
    state["relevant_docs"] = docs
    return state

def relevance_check_node(state):
    query = state["question"].lower()
    filtered = [doc for doc in state["relevant_docs"] if any(word in doc.page_content.lower() for word in query.split())]
    state["relevant_docs"] = filtered
    return state

def refine_node(state):
    question = state["question"].strip().lower()
    match = re.match(r"is\s+(\w+)\s+(?:a|an)?\s*([\w\s\-]+)\??", question)
    if match:
        const, pred = match.groups()
        pred = pred.strip().replace(" ", "_")
        state["refined_query"] = f"{pred}({const})"
    else:
        state["refined_query"] = "invalid(format)"
    return state

def inference_node(state):
    try:
        pred, const = state["refined_query"].strip(")?").split("(")
        const = const.strip()
        result = prove(pred, const)
        state["inference_result"] = f"{pred}({const}) → {result}"
    except:
        state["inference_result"] = "Invalid query format"
    return state


In [14]:
# 3. Build LangGraph

from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph

class GraphState(TypedDict):
    question: str
    relevant_docs: List[Document]
    refined_query: str
    inference_result: Optional[str]

graph = StateGraph(GraphState)


graph.add_node("retrieve", retrieve_node)
graph.add_node("filter", relevance_check_node)
graph.add_node("refine", refine_node)
graph.add_node("infer", inference_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "filter")
graph.add_edge("filter", "refine")
graph.add_edge("refine", "infer")
graph.set_finish_point("infer")

app = graph.compile()

# --- Run test queries ---
test_questions = [
    "Is cat a mammal?",
    "Is sparrow a bird?",
    "Is cat a bird?",
    "Is sparrow warm blooded?",
    "Is eagle a mammal?",
]

print("Task 9: LangGraph + Symbolic Reasoning Results\n")
for q in test_questions:
    result = app.invoke(create_state(q))
    print("="*60)
    print("Q:", q)
    print("→", result['inference_result'])


Task 9: LangGraph + Symbolic Reasoning Results

Q: Is cat a mammal?
→ mammal(cat) → True
Q: Is sparrow a bird?
→ bird(sparrow) → True
Q: Is cat a bird?
→ bird(cat) → False
Q: Is sparrow warm blooded?
→ warm_blooded(sparrow) → False
Q: Is eagle a mammal?
→ mammal(eagle) → False
